In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
import nltk

In [4]:
def calculate(gcs): #所以gcs要是string

    
    df = pd.read_csv(gcs) 

    df['start_time'] = pd.to_numeric(df['start_time'])
    df['end_time'] = pd.to_numeric(df['end_time'])

    all_sentences001 = []
    for t in range(0,480):
        sentences = []
        for index, rows in df.loc[(df["start_time"]>=t) & (df["start_time"]<=(t+10))].iterrows():
            sentences.append(rows["words"])
        all_sentences001.append(" ".join(sentences))

    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    SBERT = []

    def get_cosine_similarity(feature_vec_1, feature_vec_2):    
        return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
    # Two lists of sentences
    for i in range(0,len(all_sentences001)-1):
        #Compute embedding for both lists
        embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
        embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)

        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        # convert from PyTorch tensor to numpy array
        SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
    ######
    
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    for t in range(0,480):
        sentences = []
        for index, rows in df.loc[(df["start_time"]>=t) & (df["start_time"]<=(t+10))].iterrows():
            if rows["words"].lower() not in stops:
                sentences.append(rows["words"].lower())
        all_sentences001.append(" ".join(sentences))
    
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')

    stops = set(stopwords.words('english'))
    all_sentences_wu_sum = []
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        wu_pulmer_sum = 0
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):
                        for u in range(0, syns2_num):
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                            timeframe_pairs_count += 1
                else:
                    continue
        if timeframe_pairs_count != 0:
            all_sentences_wu_sum.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
        else: 
            all_sentences_wu_sum.append(wu_pulmer_sum)
        #print(all_sentences_wu_sum)
        wu_difference = []
        for x in range(len(all_sentences_wu_sum)-1):
            wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])

參數表:
all_sentences001: 所有的句子整合(沒有刪掉stop words)

